## ДЗ 2
#### Гордеев Никита

Я выбрал датасет отзывов на цифровые музыкальные релизы.

In [1]:
import pandas as pd
import gzip
import re
import editdistance
from collections import Counter
from nltk.corpus import stopwords

stop = stopwords.words('english')

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('reviews_Digital_Music_5.json.gz')

In [187]:
# Вот такие здесь данные, нам понадобятся только тексты и идентификаторы товара (asin), 
# чтобы понимать, какие из отзывов относятся к одному и тому же альбому или песне.

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A3EBHHCZO6V2A4,5555991584,"Amaranth ""music fan""","[3, 3]","It's hard to believe ""Memory of Trees"" came ou...",5.0,Enya's last great album,1158019200,"09 12, 2006"
1,AZPWAXJG9OJXV,5555991584,bethtexas,"[0, 0]","A clasically-styled and introverted album, Mem...",5.0,Enya at her most elegant,991526400,"06 3, 2001"
2,A38IRL0X2T4DPF,5555991584,bob turnley,"[2, 2]",I never thought Enya would reach the sublime h...,5.0,The best so far,1058140800,"07 14, 2003"
3,A22IK3I6U76GX0,5555991584,Calle,"[1, 1]",This is the third review of an irish album I w...,5.0,Ireland produces good music.,957312000,"05 3, 2000"
4,A1AISPOIIHTHXX,5555991584,"Cloud ""...""","[1, 1]","Enya, despite being a successful recording art...",4.0,4.5; music to dream to,1200528000,"01 17, 2008"
...,...,...,...,...,...,...,...,...,...
64701,A1PQ1PESSO8CMO,B00KILDVEI,Ginger Christmas,"[0, 0]",I like the reggae sound a lot in this song. I ...,4.0,Cool song,1403568000,"06 24, 2014"
64702,A120RH58WVY4W6,B00KILDVEI,"Kelly Dunwell ""avid reader""","[0, 0]",I first heard this on Sirius and had to have i...,5.0,Great Song,1404864000,"07 9, 2014"
64703,A19VJ2IQLO50G0,B00KILDVEI,melinda,"[0, 1]","I absolutely love this song, it downloaded fin...",5.0,Five Stars,1405209600,"07 13, 2014"
64704,AUDSM2CTLLW1Q,B00KILDVEI,Patrick L. Randall,"[0, 0]","Reggae, island beats aren't really my cup of t...",3.0,Well-crafted song,1404864000,"07 9, 2014"


In [2]:
ids = df['asin'].to_list()
reviews = df['reviewText'].tolist()

Допустим, мы музыкальные менеджеры и исследуем, что о каких альбомах/песнях люди говорят.

Названия песен и альбомов (и вообще предметов искусства) проигрывают в удобности обнаружения условным моделям телефонов, названиям стран или именам, потому что чаще всего представляют собой сочетания обычных слов: фразы, клаузы какой угодно длины; при том могут содержать в себе вложенные сущности, будь то имя, город или вымышленная организация а-ля Feel Good Inc.

Из-за этого мне кажется, что для обнаружения NER в нашем случае надёжнее опираться на формальные признаки. Какими они могут быть?

- несогласованность предиката с левым соседом типа Wild Thoughts is best song I ever heard
- кавычки - самый показательный!
- соседние слова типа song, track, album, CD и т. д.

Заглавные буквы тоже часто помогают, тем более в западной традиции все слова в названии принято начинать с заглавной буквы. Из-за моды на lowercase в названиях, манеры написания у ревьюверов (всё с маленькой буквы, капс или рандомные озаглавленные слова) и других именованных сущностей, этот критерий становится менее показательным, но можно попробовать.

Будем опираться на регистр и кавычки.

In [3]:
def basic_searching(text):
    candidates = []
    first_candidates = re.findall('''(?:\"(.+?)\"| \'(.+?)\' |&quot;(.+?)&quot;)''', text)
    for item in first_candidates:
        for token in item:
            if token != '' and token not in candidates:
                candidates.append(token)
    second_candidates = re.findall('''(([A-Z][a-z']+(?: |))+)''', text)
    for item in second_candidates:
        token = item[0]
        if token not in candidates:
            candidates.append(token)
    for i in range(len(candidates)):
        if candidates[i].endswith((' ', '"', "'", ',', '.')):
            candidates[i] = candidates[i][:-1]
        if candidates[i].endswith("'s'"):
            candidates[i] = candidates[i][:-2]
    #от идеи со словами-помощниками я отказался, потому что всё равно нужно ограничивать строку, которую мы берём правилами выше
    #more_candidates += re.findall('''(?:songs |song |tracks |track |ballads | ballad |album |cd |CD (?:like |as |)''', text)
    return candidates

In [5]:
counters = []
product_id = ids[0]
all_candidates = []
for i in range(1000):
    if ids[i] == product_id:
        candidates = basic_searching(reviews[i])
        for candidate in candidates:
            all_candidates.append(candidate.lower())
    else:
        product_id = ids[i]
        counters.append(Counter(all_candidates))
        all_candidates = basic_searching(reviews[i])        

Проверить, какие из предполагаемых NER подходят нам можно, поискав их в музыкальном каталоге. Возможностей для этого много (сервисы Discogs, MusicBrainz, Genius и другие), мы попробуем API Spotify. Сначала определимся, какому исполнителю принадлежит продукт с отзывами (их можно отфильтровать по asin ревью). Точное имя исполнителя упростит дальнейшие поиски.

Далее будем искать найденные "NER" в Spotify уже с известным артистом и, применяя расстояние Левенштейна, выбирать наиболее вероятные релизы, которые упоминаются в отзывах.

In [4]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy import util
import conf

cid = conf.S_TOKEN
secret = conf.S_SECRET
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [120]:
metadata = {}
for c in counters[:10]:
    artist_candidates = []
    for ner in c.keys():
        results = sp.search(q=ner, type='track')
        for i in results['tracks']['items']:
            artist = i['artists'][0]['name']
            if artist.lower() in c.keys():
                artist_candidates.append(artist)
    artist_candidates = Counter(artist_candidates)
    artist = max(artist_candidates, key=artist_candidates.get)
    print(artist, 'is chosen')
    disco = []
    for ner in c.keys():
        results = sp.search(q=artist + ' ' + ner, type='track')
        song_candidates = {}
        album_candidates = {}
        for i in results['tracks']['items']:
            if i['artists'][0]['name'].lower() == artist.lower():
                album = i['album']['name']
                song = i['name']
                album_edit_distance = editdistance.eval(ner, album.lower())
                song_edit_distance = editdistance.eval(ner, song.lower())
                album_candidates.update({album: album_edit_distance})
                song_candidates.update({song: song_edit_distance})
        if album_candidates != {}:
            album_choice = min(album_candidates, key=album_candidates.get)
            song_choice = min(song_candidates, key=song_candidates.get)
            if album_candidates[album_choice] <= song_candidates[song_choice]:
                choice = album_choice
                item_type = 'album'
            else:
                choice = song_choice
                item_type = 'song'
            if [choice, item_type] not in disco:
                disco.append([choice, item_type])
                if choice.lower().startswith('the '): #the часто опускают
                    disco.append([choice[4:], item_type])
            print(artist, choice, item_type, sep=' - ')
    metadata.update({artist: disco})

Lester Flatt is chosen
Lester Flatt - Foggy Mountain Gospel - album
Lester Flatt - Is There Room for Me - song
Lester Flatt - If I Should Wander Back Tonight - song
Lester Flatt - Is It Too Late Now? - song
Lester Flatt - Foggy Mountain Gospel - album
Lester Flatt - Cabin On The Hill - song
Lester Flatt - Bluegrass Super Hits - album
Lester Flatt - I've Lost You - song
Lester Flatt - Old Salty Dog Blues - song
Lester Flatt - Down The Road - song
Lester Flatt - Songs Of The Famous Carter Family - album
Lester Flatt - Old Salty Dog Blues - song
Lester Flatt - Bluegrass U.S.A. - album
Lester Flatt - Is It Too Late Now? - song
Lester Flatt - Doin' My Time - song
Lester Flatt - Maggie's Farm - song
Lester Flatt - Sally Goodin' - Live - song
Lester Flatt - Old Salty Dog Blues - song
Lester Flatt - The One And Only - album
Lester Flatt - Lester 'N' Mac - album
Lester Flatt - Orange Blossom Special - song
Lester Flatt - Down The Road - song
Lester Flatt - Reuben - song
Lester Flatt - Is There 

Boogie Down Productions - T' Cha - T' Cha - song
Boogie Down Productions - I'm Still #1 - song
Boogie Down Productions - Super Hoe - album
Boogie Down Productions - I'm Still #1 - song
Boogie Down Productions - 13 and Good - song
Boogie Down Productions - Criminal Minded (Deluxe) - album
Boogie Down Productions - Ya Slippin' - song
Boogie Down Productions - Edutainment - album
Boogie Down Productions - Jimmy - song
Boogie Down Productions - I'm Still #1 - song
Boogie Down Productions - You Must Learn - song
Boogie Down Productions - Illegal Business - song
Boogie Down Productions - T' Cha - T' Cha - song
Boogie Down Productions - We in There - song
Boogie Down Productions - South Bronx - song
Boogie Down Productions - Why Is That? - song
Boogie Down Productions - My Philosophy - song
Boogie Down Productions - Edutainment - album
Boogie Down Productions - A Retrospective - album
Boogie Down Productions - Jah Rulez - song
Boogie Down Productions - Jah Rulez - song
Boogie Down Productions

Заново проходимся по текстам ревью, уже выискивая названия по нашему словарю метаданных. Тексты предобрабатываем: удаляем стоп-слова и знаки препинания. Собираем левых и правых соседей каждой найденной сущности в словари коллокаций.

Если NER встречается в варианте с the, мы берём его; иначе - вариант без артикля.

В этом варианте поиска мы не можем быть уверены, что соберём все упоминания продуктов: альтернативные названия, вариации в пунктуации и банальные ошибки/описки не попадут в результаты. Но с нашим подходом единственный выход, который я вижу, - проходиться рамкой по всему отзыву, сверяя все строки с поисковым запросом из словаря метаданных, и учитывать возможность погрешности расстоянием Левенштейна. Но тогда при поиске коротких NER типа Ocean или Celts попадёт много мусора.

В целом, вроде неплохо получилось (много совпадений).

In [76]:
def preprocess(text):

    text = re.sub('''[.'",!?:;()]''', '', text)
    text = re.sub('''&quot''', '', text)
    for word in stop:
        text = text.replace(' '+ word + ' ', ' ')
    return text


In [209]:
product_id = ids[0]
left_collocations = {}
right_collocations = {}
artists = list(metadata.keys())
artist_id = 0
for j in range(len(ids)):
    if ids[j] == product_id:
        text = reviews[j].lower()
        for i in metadata[artists[artist_id]]:
            if i[0].lower() in text:
                print(i[0])
                parts = text.split(i[0].lower())
                parts[0] = preprocess(parts[0])
                left = parts[0].split()
                if len(left) > 5:
                    left = left[-5:]
                print(left)
                parts[1] = preprocess(parts[1])
                right = parts[1].split()
                if len(right) > 5:
                    right = right[:5]
                print(right)
                text = text.replace(i[0].lower(), '')
                if i[0] not in left_collocations:
                    left_collocations.update({i[0]: []})
                if left != []:
                    left_collocations[i[0]].append(left[-1])
                if i[0] not in right_collocations:
                    right_collocations.update({i[0]: []})
                if right != []:
                    right_collocations[i[0]].append(right[0])
    else:
        product_id = ids[j]
        artist_id += 1
    if artist_id > len(artists) - 1:
        break

Memory of Trees
['its', 'hard', 'believe']
['came', '11', 'years', 'agoit', 'held']
Amarantine
['last', 'great', 'album', 'new', 'age/pop']
['day', 'without', 'rain', 'back', '1995enya']
Pax Deorum
['melancholybittersweetfrom', 'opening', 'title', 'song', 'elegaic&majestic']
['sounds', 'like', 'requiem', 'massit', 'dark']
Anywhere Is
['spirituality&sensualityi', 'dont', 'find', 'disconcerting', 'all']
['hopeful', 'songlooking', 'possibilitieshope', 'place', 'lovebut']
Hope Has a Place
['find', 'disconcerting', 'hopeful', 'songlooking', 'possibilities']
['lovebut', 'listener', 'decide', 'romanticplatonicetcive', 'always']
On My Way Home
['romanticplatonicetcive', 'always', 'soft', 'spot', 'song']
['triumphant', 'ending', 'returnthis', 'truly', 'masterpiece']
Memory of Trees
['a', 'clasically-styled', 'introverted', 'album']
['masterpiece', 'subtlety', 'many', 'songs', 'endearing']
Watermark
['start', 'one', 'popularized', 'works', 'like']
['play', 'safe', 'youre', 'already', 'fan']
Hope

['witch', 'imo', 'best', 'song', 'cd']
Keep It Right There
['cd', 'also', 'like', 'te', 'song']
[]
Stroke You Up
['this', 'record', 'slept', 'exception', 'hit']
['feat/rkellythe', 'reason', 'could', 'think', 'flooded']
Stroke You Up
['in', 'high', 'school']
['jam', 'went', 'downtown', 'bought', 'cassette']
Changing Faces
['first', '*and*', 'last', 'time', 'listened']
['two', 'chicks', 'made']
Stroke You Up
['remember', 'joint', 'r', 'kelly', 'talking']
['anyway', 'ok', 'would', 'suggest', 'purchasing']
Stroke You Up
['i', 'first', 'bought', 'cd-single']
['back', '94', 'tape', 'came', 'wore']
Foolin' Around
['hits', 'first', 'two', 'singles', '&']
['produced', 'r', 'kelly', 'third', 'single']
Keep It Right There
['kelly', 'third', 'single', 'favorite', 'album']
['urban', 'touch', 'produced', 'one', 'favorite']
Baby Your Love
['feeling', 'love', 'movin', 'thoughts', '&']
['produced', 'heavy', 'overall', 'great', 'album']
Thoughts of You
['highlights', 'album', 'feeling', 'love', 'movin']

['years', 'ago', 'already', 'quite', 'familiar']
['really', 'got', 'hooked', 'good', 'times']
All Mixed Up
['totally', 'fallen', 'medley', 'moving', 'stereo/']
['talk', 'true', 'classic', 'years', 'every']
I'm in Touch with Your World
['got', 'tonight', 'following', 'year', 'got']
['past', 'couple', 'years', 'totally', 'fallen']
The Cars
[]
['1978']
Drive
['combined', 'hard', 'guitar-rock', 'spacey', 'keyboard-']
['n', 'new', 'wave', 'sound', 'beginning']
You're All I've Got Tonight
['wasnt', '1982', 'really', 'started', 'liking']
['following', 'year', 'got', 'past', 'couple']
Good Times Roll
['quite', 'familiar', 'really', 'got', 'hooked']
['best', 'friends', 'girl', 'wasnt', '1982']
Moving in Stereo
['couple', 'years', 'totally', 'fallen', 'medley']
['/', 'talk', 'true', 'classic', 'years']
Heartbeat City
['pop', 'enthusiasts', 'would', 'probably', 'say']
['sell', 'records', 'upon', 'release', 'real']
New Wave
['combined', 'hard', 'guitar-rock', 'spacey', 'keyboard-n']
['sound', 'beg

['including', 'awesomely', 'afos', 'tracks', 'tokyo']
['plus', 'get', 'tanglimara', 'already', 'bonus']
End
['version', 'without', 'space', 'intro', 'cold']
['bonus', 'track', 'would', 'way', 'really']
Space Age Love Song
['80s', 'isnt', 'bad', 'song', 'disk']
['perfect', 'song', 'absolutely', 'perfect']
Best Of
['stand', 'crowd', 'advice', 'skip', 'many']
['compilations', 'go', 'straight', 'one', 'get']
D.N.A.
['-', 'best', 'instrumental', 'year', 'fantastic']
['debut', 'album', 'classic', 'example', 'band']
I Ran
['instrumental', 'track', 'featuring', 'synthesizers', 'guitars9']
['far', 'awaythe', 'song', 'best', 'known']
Space Age Love Song
['self', 'titled', 'album', 'came', '19821']
['awesome', 'futuristic', 'song', 'guy', 'falling']
D.N.A.
['telling', 'story', 'man', 'spying', 'someone8']
['an', 'instrumental', 'track', 'featuring', 'synthesizers']
Don't Ask Me
['riffs', 'rule', 'cant', 'run', 'yourself3']
['a', 'strange', 'song', 'saucers4', 'messagesits']
Telecommunication
['st

['trunk', 'first', '`official', 'album', '1988']
['beat', 'playalisticand', 'rhymes', 'pizoint', 'song']
Freaky Tales
['good', 'melody', 'hit', 'song', 'album']
['almost', '10min', 'long', 'short', 'comes']
Freaky Tales
['95', 'tight', 'beat', '9+', 'minutes']
['#6', '-', '95', 'another', 'tight']
Listen
['must-have', 'rap', 'fans', 'collection', 'beginning']
['album', 'see', 'started']


IndexError: list index out of range

Не в каждом отзыве обсуждаются все песни с альбома, поэтому соседи получились довольно уникальные. Объединим все коллокации названий песен и альбомов в общие списки по исполнителям. Будем считать, что это такой entity linking и мы узнаём мнения об исполнителе в целом.

In [234]:
artist_left_collocations = {}
artist_right_collocations = {}
for i, j in metadata.items():
    tags = []
    for item in right_collocations.keys():
        for u in j:
            if item in u:
                artist = i
                for token in right_collocations[item]:
                    if token.isalpha():
                        tags.append(token)
    artist_right_collocations.update({artist: tags})
    tags = []
    for item in left_collocations.keys():
        for u in j:
            if item in u:
                artist = i
                for token in left_collocations[item]:
                    if token.isalpha():
                        tags.append(token)
    artist_left_collocations.update({artist: tags})

Выводим топ-5 левых и правых соседей для пяти выбранных исполнителей.

In [289]:
artists_to_show = ['Enya', 'Changing Faces', 'The Cars', 'A Flock Of Seagulls', 'Boogie Down Productions']
extra_stops = ['album', 'albums', 'song', 'songs', 'track', 'tracks', 'a', 'is', 'im'] #ещё почистим неинформативное

for artist in artists_to_show:
    print('_________________________', artist, '_________________________', 'LEFT:', sep='\n')
    l = Counter(artist_left_collocations[artist])
    for e in extra_stops:
        if e in l:
            l.pop(e)
    l = l.most_common(5)
    for i in l:
        print(i[0])
    print('_________________________', 'RIGHT:', sep='\n')
    r = Counter(artist_right_collocations[artist])
    for e in extra_stops:
        if e in r:
            r.pop(e)
    r = r.most_common(5)
    for i in r:
        print(i[0])

_________________________
Enya
_________________________
LEFT:
sky
heights
instrumental
us
recordings
_________________________
RIGHT:
another
sung
enya
one
day
_________________________
Changing Faces
_________________________
LEFT:
movin
hit
like
classic
truesongand
_________________________
RIGHT:
movin
still
one
written
produced
_________________________
The Cars
_________________________
LEFT:
girl
like
roll
tonight
hits
_________________________
RIGHT:
moving
youre
one
dont
first
_________________________
A Flock Of Seagulls
_________________________
LEFT:
like
hit
tunes
called
version
_________________________
RIGHT:
modern
first
drums
space
messages
_________________________
Boogie Down Productions
_________________________
LEFT:
like
means
line
get
bronx
_________________________
RIGHT:
one
stop
jimmy
dropped
many


Вот такие получились "коллокации", а точнее соседи названий релизов каждого из исполнителей. Трудно назвать их информативными, но таковые надо искать в рамках хотя бы по 5 слов слева и справа от таргета (там действительно было гораздо больше оценочных слов).

Главная сложность, как мне кажется, связана с выбранным датасетом: не понятно, что считать продуктом, какие вычленять именованные сущности. К тому же, из-за специфики отзывов на искусство (разнообразие лексики, подходов к оценке продукта и немного журналистский стиль) куча слов типа masterpiece или great получились низкочастотными, поэтому коллокации получились странные.

Ещё в последнем шаге я убирал слова album, song, track, можно было бы их приобщить к выделенным именованным сущностям и смотреть на соседей вместе с ними.

P.S. Со spacy, наверняка, получилось бы лучше...

In [294]:
import spacy 
nlp = spacy.load("en_core_web_sm")

doc = nlp(reviews[11])
for ent in doc.ents:
    print(ent.text,  ent.label_)

the past twenty-odd years DATE
Enya Brennan PERSON
New Age ORG
The Memory WORK_OF_ART
Enya PERSON
first ORDINAL
a clear fall day DATE
Latin LANGUAGE
Gold ORG
weekend DATE
Time ORG
The Memory of Trees WORK_OF_ART
Enya PERSON
nearly twenty years DATE
one CARDINAL
fifteen years DATE
The Memory of Trees WORK_OF_ART
Enya PERSON
